### PRUEBA

In [2]:
# PASO 1.1 — Montar Drive y LISTAR .zip
from google.colab import drive
drive.mount('/content/drive')

import os, glob

# Ajusta si usas otra carpeta
CANDIDATE_DIRS = [
    "/content/drive/MyDrive/DATASETS/dataset_1024_resolucion",
    "/content/drive/MyDrive/Colab Notebooks/DATASETS/dataset_1024_resolucion",
]

Mounted at /content/drive


In [3]:
BASE_DIR = None
for d in CANDIDATE_DIRS:
    if os.path.isdir(d):
        BASE_DIR = d
        break

if BASE_DIR is None:
    print("No encontré la carpeta del dataset en las rutas esperadas.")
    print("Indícame la ruta exacta. Contenido de /content/drive/MyDrive:")
    for item in sorted(os.listdir("/content/drive/MyDrive")):
        print(" -", item)
else:
    print("✅ Carpeta localizada en:", BASE_DIR)
    zips = sorted(glob.glob(os.path.join(BASE_DIR, "*.zip")))
    if not zips:
        print("No hay .zip en esa carpeta.")
    else:
        print("ZIPs encontrados:")
        for z in zips:
            print(" -", os.path.basename(z))
        v1 = [z for z in zips if "v4" in os.path.basename(z).lower()]
        SELECTED_ZIP = v1[0] if v1 else zips[0]
        print("\n➡️  Seleccionado:", os.path.basename(SELECTED_ZIP))

✅ Carpeta localizada en: /content/drive/MyDrive/Colab Notebooks/DATASETS/dataset_1024_resolucion
ZIPs encontrados:
 - dataset_arboles_v3.zip
 - dataset_arboles_v4.zip

➡️  Seleccionado: dataset_arboles_v4.zip


In [4]:
import os, shutil, zipfile

# Usa la misma variable BASE_DIR del paso anterior
ZIP_V1 = None
for f in os.listdir(BASE_DIR):
    if "v4" in f.lower() and f.endswith(".zip"):
        ZIP_V1 = os.path.join(BASE_DIR, f)
        break

if ZIP_V1 is None:
    raise FileNotFoundError("No encontré un archivo con 'v1' en su nombre dentro de la carpeta del dataset.")

print("Archivo seleccionado:", os.path.basename(ZIP_V1))

Archivo seleccionado: dataset_arboles_v4.zip


In [5]:
# Copiar al almacenamiento local de Colab
local_zip = "/content/dataset_v4.zip"
if os.path.exists(local_zip):
    os.remove(local_zip)
shutil.copy2(ZIP_V1, local_zip)
print("Copiado a:", local_zip)

Copiado a: /content/dataset_v4.zip


In [6]:
# Descomprimir en carpeta limpia
extract_dir = "/content/dataset_yolov4_l2"
if os.path.exists(extract_dir):
    shutil.rmtree(extract_dir)
os.makedirs(extract_dir, exist_ok=True)

with zipfile.ZipFile(local_zip, "r") as zf:
    zf.extractall(extract_dir)

print("Descomprimido en:", extract_dir)
print("\nContenido de primer nivel:")
for item in os.listdir(extract_dir):
    p = os.path.join(extract_dir, item)
    print(" - [DIR]" if os.path.isdir(p) else " - [FILE]", item)

Descomprimido en: /content/dataset_yolov4_l2

Contenido de primer nivel:
 - [DIR] dataset_arboles_v4


In [7]:
# Detectar la carpeta raíz del dataset descomprimido
import os

EXTRACT_DIR = "/content/dataset_yolov4_l2"  # carpeta donde se descomprimió el ZIP

def pick_root(extract_dir):
    """Si solo hay una subcarpeta dentro, entrar en ella; si no, usar extract_dir."""
    items = [i for i in os.listdir(extract_dir) if not i.startswith(".")]
    if len(items) == 1 and os.path.isdir(os.path.join(extract_dir, items[0])):
        return os.path.join(extract_dir, items[0])
    return extract_dir

ROOT = pick_root(EXTRACT_DIR)

print("Raíz detectada:", ROOT)
print("\nContenido dentro de la raíz:")
for i in os.listdir(ROOT):
    print(" -", i)

Raíz detectada: /content/dataset_yolov4_l2/dataset_arboles_v4

Contenido dentro de la raíz:
 - data.yaml
 - images
 - labels


In [8]:
# Buscar y leer data.yaml (si existe)
import os, glob, yaml

yaml_candidates = []
for pattern in ("*.yaml", "*.yml"):
    yaml_candidates += glob.glob(os.path.join(ROOT, pattern))

if not yaml_candidates:
    print("No se encontró data.yaml ni data.yml en la raíz del dataset.")
else:
    print("Archivos YAML encontrados:")
    for y in yaml_candidates:
        print(" -", os.path.basename(y))

    try:
        # Tomar preferentemente el que se llame data.yaml
        preferred = [y for y in yaml_candidates if "data" in os.path.basename(y).lower()]
        preferred = preferred[0] if preferred else yaml_candidates[0]
        print("\n Leyendo:", os.path.basename(preferred))

        with open(preferred, "r", encoding="utf-8") as f:
            yaml_info = yaml.safe_load(f)

        print("\nContenido del YAML:\n")
        print(yaml_info)
    except Exception as e:
        print("Error al leer el YAML:", e)

Archivos YAML encontrados:
 - data.yaml

 Leyendo: data.yaml

Contenido del YAML:

{'path': '/content/drive/MyDrive/Colab Notebooks/DATASETS/dataset_1024_resolucion/', 'train': 'images/train', 'val': 'images/val', 'test': 'images/test', 'names': {0: 'arbol'}}


In [9]:
import os

splits = ["train", "val", "test"]
layout = None
paths = {}

# Caso A: images/train y labels/train
if os.path.isdir(os.path.join(ROOT, "images")) and os.path.isdir(os.path.join(ROOT, "labels")):
    layout = "images_split"
    for s in splits:
        paths[s] = {
            "images": os.path.join(ROOT, "images", s),
            "labels": os.path.join(ROOT, "labels", s)
        }

# Caso B: train/images y val/images
if layout is None:
    ok = True
    temp = {}
    for s in splits:
        si = os.path.join(ROOT, s, "images")
        sl = os.path.join(ROOT, s, "labels")
        if not (os.path.isdir(si) and os.path.isdir(sl)):
            ok = False
            break
        temp[s] = {"images": si, "labels": sl}
    if ok:
        layout = "split_images"
        paths = temp

if layout:
    print("✅ Estructura detectada:", layout)
    for s, d in paths.items():
        print(f"\n{s.upper()}:")
        print("  images →", d["images"])
        print("  labels →", d["labels"])
else:
    print("No se detectó estructura YOLO válida.")

✅ Estructura detectada: images_split

TRAIN:
  images → /content/dataset_yolov4_l2/dataset_arboles_v4/images/train
  labels → /content/dataset_yolov4_l2/dataset_arboles_v4/labels/train

VAL:
  images → /content/dataset_yolov4_l2/dataset_arboles_v4/images/val
  labels → /content/dataset_yolov4_l2/dataset_arboles_v4/labels/val

TEST:
  images → /content/dataset_yolov4_l2/dataset_arboles_v4/images/test
  labels → /content/dataset_yolov4_l2/dataset_arboles_v4/labels/test


In [10]:
# PASO 2.4 — Conteo de imágenes y etiquetas por split
import os

def count_files(img_dir, lbl_dir):
    imgs = [f for f in os.listdir(img_dir) if f.lower().endswith(('.jpg', '.png', '.jpeg'))]
    lbls = [f for f in os.listdir(lbl_dir) if f.lower().endswith('.txt')]
    print(f"    Imágenes: {len(imgs)}  |    Etiquetas: {len(lbls)}")
    if len(imgs) != len(lbls):
        print("  Diferencia detectada entre número de imágenes y etiquetas.")
        # Mostrar ejemplos de archivos sin pareja
        stem_imgs = {os.path.splitext(f)[0] for f in imgs}
        stem_lbls = {os.path.splitext(f)[0] for f in lbls}
        solo_imgs = sorted(list(stem_imgs - stem_lbls))[:5]
        solo_lbls = sorted(list(stem_lbls - stem_imgs))[:5]
        if solo_imgs:
            print("   - Faltan etiquetas para:", solo_imgs)
        if solo_lbls:
            print("   - Faltan imágenes para:", solo_lbls)
    return len(imgs), len(lbls)

if 'paths' in globals() and paths:
    print("\n===  RESUMEN ===")
    for s, d in paths.items():
        print(f"\n[{s.upper()}]")
        if os.path.isdir(d["images"]) and os.path.isdir(d["labels"]):
            count_files(d["images"], d["labels"])
        else:
            print("Falta carpeta:", d)
else:
    print("No se detectaron rutas válidas en el paso anterior (2.3).")


===  RESUMEN ===

[TRAIN]
    Imágenes: 8645  |    Etiquetas: 8645

[VAL]
    Imágenes: 2470  |    Etiquetas: 2470

[TEST]
    Imágenes: 1235  |    Etiquetas: 1235


In [11]:
# Instalar y verificar YOLOv8 (Ultralytics)
!pip install ultralytics --upgrade --quiet

from ultralytics import YOLO
import ultralytics

print("Ultralytics instalado correctamente.")
print("Versión:", ultralytics.__version__)

# Verificar GPU disponible
import torch
print("GPU detectada:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No hay GPU activa")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 62.3 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics instalado correctamente.
Versión: 8.3.223
GPU detectada: NVIDIA A100-SXM4-80GB


In [16]:
import os, yaml

yaml_path = os.path.join(ROOT, "data.yaml")

# Rutas relativas desde la raíz del dataset
train_path = os.path.relpath(paths["train"]["images"], ROOT)
val_path   = os.path.relpath(paths["val"]["images"], ROOT)
test_path  = os.path.relpath(paths["test"]["images"], ROOT)

# Definir las clases (ajusta si tienes más de una)
data_config = {
    "path": ROOT,
    "train": train_path,
    "val": val_path,
    "test": test_path,
    "names": ["arbol"],  # puedes añadir más clases aquí
}

with open(yaml_path, "w", encoding="utf-8") as f:
    yaml.dump(data_config, f, sort_keys=False, allow_unicode=True)

print("✅ Archivo data.yaml creado correctamente en:", yaml_path)
print("\nContenido del data.yaml:\n")
print(yaml.dump(data_config, sort_keys=False, allow_unicode=True))

✅ Archivo data.yaml creado correctamente en: /content/dataset_yolov4_l2/dataset_arboles_v4/data.yaml

Contenido del data.yaml:

path: /content/dataset_yolov4_l2/dataset_arboles_v4
train: images/train
val: images/val
test: images/test
names:
- arbol



In [17]:
# Entrenamiento YOLOv8l - Precision Pack (AdamW + batch=12)
from ultralytics import YOLO
import os

# Ruta del YAML de tu dataset
yaml_path = os.path.join(ROOT, "data.yaml")

# Carga el modelo base YOLOv8l
model = YOLO("yolov8l.pt")

# Entrenamiento enfocado en PRECISIÓN
results = model.train(
    data=yaml_path,
    epochs=120,
    imgsz=1024,
    batch=12,               # A100 lo aguanta sin problema
    device=0,
    workers=8,
    seed=42,
    project="/content/runs",
    name="yolov8l_1024_precision_pack_b12",

    # Optimizador y LR ajustado al batch
    optimizer="AdamW",
    lr0=0.0006,             # escalado para batch=12
    weight_decay=0.0003,
    cos_lr=True,
    lrf=0.01,

    # Estabilidad y augmentations sobrias (satelital)
    patience=60,
    close_mosaic=12,
    cache=True,
    hsv_h=0.015, hsv_s=0.5, hsv_v=0.35,
    fliplr=0.5, flipud=0.0,
    degrees=0.0, translate=0.05, scale=0.35, shear=0.0,

    # Guardado y validación
    save=True,
    save_period=10,
    plots=True,
    val=True
)

print("\n✅ Entrenamiento lanzado/finalizado.")
print("Carpeta de resultados:", results.save_dir)
print("Pesos mejores (best.pt):", os.path.join(str(results.save_dir), "weights", "best.pt"))
print("Últimos pesos (last.pt):", os.path.join(str(results.save_dir), "weights", "last.pt"))

Ultralytics 8.3.223 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (NVIDIA A100-SXM4-80GB, 81222MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=12, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=12, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/content/dataset_yolov4_l2/dataset_arboles_v4/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=120, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.5, hsv_v=0.35, imgsz=1024, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0006, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8l.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8l_1024_precision_pack_b123, nbs=64, nms=False, opset=None, optimize=False, optimizer=AdamW, 

In [18]:
!ls -lt /content/runs


total 12
drwxr-xr-x 3 root root 4096 Nov  2 22:35 yolov8l_1024_precision_pack_b123
drwxr-xr-x 3 root root 4096 Nov  2 15:22 yolov8l_1024_precision_pack_b122
drwxr-xr-x 3 root root 4096 Nov  2 15:22 yolov8l_1024_precision_pack_b12


In [21]:
!cp -r /content/runs/yolov8l_1024_precision_pack_b123 /content/drive/MyDrive/yolo_runs_backup_versionL_adams_lr_v1/

In [22]:
from IPython.display import Image, display
import pandas as pd
import os

# Ruta a tu copia en Drive
run_path = "/content/drive/MyDrive/yolo_runs_backup_versionL_adams_lr_v1"

csv_path = f"{run_path}/results.csv"
df = pd.read_csv(csv_path)

print("Columnas disponibles:", df.columns.tolist(), "\n")
print("Últimas métricas del entrenamiento:")
print(df.tail(3))  #

Columnas disponibles: ['epoch', 'time', 'train/box_loss', 'train/cls_loss', 'train/dfl_loss', 'metrics/precision(B)', 'metrics/recall(B)', 'metrics/mAP50(B)', 'metrics/mAP50-95(B)', 'val/box_loss', 'val/cls_loss', 'val/dfl_loss', 'lr/pg0', 'lr/pg1', 'lr/pg2'] 

Últimas métricas del entrenamiento:
     epoch     time  train/box_loss  train/cls_loss  train/dfl_loss  \
117    118  25132.9         0.85662         0.41807         1.09932   
118    119  25345.2         0.85173         0.41493         1.09841   
119    120  25558.1         0.85690         0.41705         1.09983   

     metrics/precision(B)  metrics/recall(B)  metrics/mAP50(B)  \
117               0.77865            0.68011           0.75160   
118               0.77871            0.67972           0.75155   
119               0.77941            0.67925           0.75128   

     metrics/mAP50-95(B)  val/box_loss  val/cls_loss  val/dfl_loss    lr/pg0  \
117              0.46530       1.54422       0.95653       1.76051  0.00

In [23]:
precision_final = df["metrics/precision(B)"].iloc[-1]
recall_final = df["metrics/recall(B)"].iloc[-1]
map50_final = df["metrics/mAP50(B)"].iloc[-1]
map95_final = df["metrics/mAP50-95(B)"].iloc[-1]

print(f"Precisión final: {precision_final:.3f}")
print(f"Recall final: {recall_final:.3f}")
print(f"mAP@50: {map50_final:.3f}")
print(f"mAP@50-95: {map95_final:.3f}")

Precisión final: 0.779
Recall final: 0.679
mAP@50: 0.751
mAP@50-95: 0.466
